In [87]:
import pandas as pd
import numpy as np
from scipy.signal import butter,filtfilt,find_peaks
import plotly.graph_objects as go


In [101]:
df_acc = pd.read_csv('../Data/Data_Nils/10_schritte/accelerometer.csv')
df_acc

,Time (s),Linear Acceleration x (m/s^2),Linear Acceleration y (m/s^2),Linear Acceleration z (m/s^2),Absolute acceleration (m/s^2)
0,0.048417,0.000000,0.000000,0.000000,0.000000
1,0.053468,-0.010919,0.015101,0.052997,0.056177
2,0.058519,0.011785,0.018143,0.005239,0.022260
3,0.063569,-0.005319,0.005607,0.041769,0.042478
4,0.068620,0.086524,-0.026251,0.073637,0.116610
...,...,...,...,...,...
4840,24.493310,0.074038,0.004699,-0.118103,0.139471
4841,24.498361,0.073943,-0.028071,-0.125479,0.148325
4842,24.503412,0.135396,-0.152538,-0.151891,0.254305
4843,24.508451,0.101453,-0.134591,-0.285137,0.331226


In [103]:
readings_np = df_acc.to_numpy()


array([[ 4.84173520e-02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 5.34679770e-02, -1.09185278e-02,  1.51009560e-02,
         5.29966354e-02,  5.61773663e-02],
       [ 5.85186020e-02,  1.17849559e-02,  1.81427002e-02,
         5.23948669e-03,  2.22597164e-02],
       ...,
       [ 2.45034119e+01,  1.35396481e-01, -1.52537942e-01,
        -1.51890755e-01,  2.54304605e-01],
       [ 2.45084515e+01,  1.01452619e-01, -1.34590626e-01,
        -2.85137176e-01,  3.31225723e-01],
       [ 2.45135110e+01,  4.72188890e-02, -1.20103598e-01,
        -3.50919724e-01,  3.73897245e-01]])

In [104]:
def butter_lowpass_filter(data):
    cutoff = 2
    fs = 200
    order = 2
    normal_cutoff = cutoff / 100
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

In [105]:
filtered_readings_np = np.apply_along_axis(butter_lowpass_filter, 0, readings_np[:, 1:])

In [108]:
filtered_readings_np = np.c_[readings_np[:, 0], filtered_readings_np]

In [109]:
# Filter the data, and plot both the original and filtered signals.
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 1],
            line =  dict(shape =  'spline' ),
            name = 'x signal')
            )
fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 2],
            line =  dict(shape =  'spline' ),
            name = 'y signal')
            )
fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 3],
            line =  dict(shape =  'spline' ),
            name = 'z signal')
            )
fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 0],
            line =  dict(shape =  'spline' ),
            name = 'time signal')
            )
fig.show()

In [110]:
def helper_find_peaks(data):
    ret, _ = find_peaks(data, height=0.1)
    return ret

In [111]:
# find peaks in absolute acceleration -> resulting in steps
peaks = []

for i in range(1, 4):
    peaks.append(helper_find_peaks(filtered_readings_np[:, i]))

peaks

[array([ 197, 1780, 1879, 2175, 2436, 2715, 2811, 3072, 3387], dtype=int64),
 array([1827, 2040, 2214, 2506, 2663, 2815, 2957, 3092, 3243, 3595],
       dtype=int64),
 array([1906, 2081, 2244, 2405, 2559, 2693, 2838, 2997, 3143, 3274],
       dtype=int64)]

# Show Peaks In Y-Axis

In [112]:
filtered_df = pd.DataFrame(filtered_readings_np, columns=['time', 'x', 'y', 'z', 'abs'])

In [128]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = filtered_df['y'],
            line =  dict(shape =  'spline' ),
            name = 'x signal')
            )
fig.add_trace(go.Scatter(
            x = peaks[1],
            y = filtered_df.loc[peaks[1]]['y'],
            mode =  'markers',
            name = 'peaks')
            )